In [2]:
import numpy as np
import pandas as pd

import os

from wide_and_deep.utils import prepare_dataset

In [ ]:
---
---

In [3]:
### Load the data set from '.data/audult_data.csv'
raw_df = pd.read_csv(os.path.join(str(os.getcwd()), 'data/adult_data.csv'))

raw_df['income_label'] = raw_df['income_bracket'].apply(lambda x: 1 if '>50K' in x or '>50K.' in x else 0)
print('Target : {} / Non-Target : {}'.format(len(raw_df[raw_df.income_label == 1]), len(raw_df[raw_df.income_label != 1])))

# For Wide and Crossed network (for memorization)
wide_cols = ['age','hours_per_week','education', 'relationship','workclass',
             'occupation','native_country','gender']
crossed_cols = (['education', 'occupation'], ['native_country', 'occupation']) # pair-combination 

# For Deep network (embedding + continuous) (for generalization)
continuous_cols = ["age","hours_per_week"]

embeddings_cols = [('education',10), ('relationship',8), ('workclass',10),
                    ('occupation',10),('native_country',12)] # column name / embedding dim.

Target : 11687 / Non-Target : 37155


In [4]:
out_dataset = prepare_dataset(raw_df, wide_cols, crossed_cols, embeddings_cols, 
                              continuous_cols, target = 'income_label', scale = True, seed = 1)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


return key values dict_keys(['train_set', 'test_set', 'embeddings_input', 'deep_column_idx', 'encoding_dict'])
